# Ethnicity

lamindb provides access to the following public protein ontologies through [lnschema-bionty](https://github.com/laminlabs/lnschema-bionty):

1. [Human Ancestry Ontology](https://github.com/EBISPOT/hancestro)

Here we show how to access and search Ethnicity ontologies to standardize new data.

## Setup

In [ ]:
!lamin init --storage ./test-ethnicity --schema bionty

In [ ]:
import lnschema_bionty as lb
import pandas as pd

## PublicOntology objects

Let us create a public ontology accessor with {meth}`~docs:lnschema_bionty.dev.BioRegistry.public`, which chooses a default public ontology source from {class}`~docs:lnschema_bionty.PublicSource`.
It's a [PublicOntology](https://lamin.ai/docs/lnschema_bionty.dev.publicontology) object, which you can think about as a public registry:

In [ ]:
ethnicitys = lb.Ethnicity.public(organism="human")
ethnicitys

As for registries, you can export the ontology as a `DataFrame`:

In [ ]:
df = ethnicitys.df()
df.head()

Unlike registries, you can also export it as a Pronto object via `public.ontology`.

## Look up terms

As for registries, terms can be looked up with auto-complete:

In [ ]:
lookup = ethnicitys.lookup()

The `.` accessor provides normalized terms (lower case, only contains alphanumeric characters and underscores):

In [ ]:
lookup.american

To look up the exact original strings, convert the lookup object to dict and use the `[]` accessor:

In [ ]:
lookup_dict = lookup.dict()
lookup_dict["American"]

By default, the `name` field is used to generate lookup keys. You can specify another field to look up:

In [ ]:
lookup = ethnicitys.lookup(ethnicitys.ontology_id)

In [ ]:
lookup.hancestro_0463

## Search terms

Search behaves in the same way as it does for registries:

In [ ]:
ethnicitys.search("American").head(3)

By default, search also covers synonyms:

In [ ]:
ethnicitys.search("Caucasian").head(3)

You can turn this off synonym by passing `synonyms_field=None`:

In [ ]:
ethnicitys.search("Caucasian", synonyms_field=None).head(3)

Search another field (default is `.name`):

In [ ]:
ethnicitys.search(
    "General characterisation of Ancestry of a population",
    field=ethnicitys.definition,
).head()

## Standardize Ethnicity identifiers

Let us generate a `DataFrame` that stores a number of Ethnicity identifiers, some of which corrupted:

In [ ]:
df_orig = pd.DataFrame(
    index=[
        "Mende"
        "European"
        "South Asian"
        "Arab"
        "This ethnicity does not exist",
    ]
)
df_orig

We can check whether any of our values are validated against the ontology reference:

In [ ]:
validated = ethnicitys.validate(df_orig.index, ethnicitys.name)
df_orig.index[~validated]

## Ontology source versions

For any given entity, we can choose from a number of versions:

In [ ]:
lb.PublicSource.filter(entity="Ethnicity").df()

When instantiating a Bionty object, we can choose a source or version:

In [ ]:
public_source = lb.PublicSource.filter(
    source="hancestro", version="3.0", organism="human"
).one()
ethnicitys= lb.Ethnicity.public(public_source=public_source)
ethnicitys

The currently used ontologies can be displayed using:

In [ ]:
lb.PublicSource.filter(currently_used=True).df()

In [ ]:
!lamin delete --force test-ethnicity
!rm -r test-ethnicity